In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os

In [25]:
!pip install pymongo

In [8]:
# File to Load
gun_violence = "Resources/gun-violence-data_01-2013_03-2018.csv"
full_moon = "Resources/full_moon.csv"

In [27]:
#view gun violence data
gun_violence_df = pd.read_csv(gun_violence)
gun_violence_df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [10]:
#Get row and column count
gun_violence_df.shape

(239677, 29)

In [11]:
#clean data before we merge data
gun_violence_df = gun_violence_df.drop(columns=['incident_id', 'incident_url', 'source_url', 'participant_age',\
                                               'latitude','location_description','incident_url_fields_missing',\
                                                'congressional_district','gun_stolen','gun_type','location_description',\
                                                'longitude','n_guns_involved','notes','participant_age','participant_age_group',\
                                                'participant_gender','participant_name','participant_relationship','participant_status',\
                                                'participant_type','sources','state_house_district','state_senate_district','address' ])
gun_violence_df.shape

(239677, 6)

In [12]:
gun_violence_df['date']=pd.to_datetime(gun_violence_df['date'])
gun_violence_df.dtypes

date                        datetime64[ns]
state                               object
city_or_county                      object
n_killed                             int64
n_injured                            int64
incident_characteristics            object
dtype: object

In [13]:
#view full moon data
full_moon_df = pd.read_csv(full_moon)
full_moon_df.head()

,Day,Date old,Month,Day.1,Year,Concatenate,DATE,Time
0,Monday,15 January 1900,1,15,1900,1/15/1900,1/15/1900,08:07:30 pm
1,Wednesday,14 February 1900,2,14,1900,2/14/1900,2/14/1900,02:50:12 pm
2,Friday,16 March 1900,3,16,1900,3/16/1900,3/16/1900,09:11:48 am
3,Sunday,15 April 1900,4,15,1900,4/15/1900,4/15/1900,02:02:06 am
4,Monday,14 May 1900,5,14,1900,5/14/1900,5/14/1900,04:36:36 pm


In [14]:
#rename DATE column in full_moon_data_df to match column in gun_violence_data_df
full_moon_df.rename(columns = {'DATE':'date'}, inplace = True) 
full_moon_df.head()

,Day,Date old,Month,Day.1,Year,Concatenate,date,Time
0,Monday,15 January 1900,1,15,1900,1/15/1900,1/15/1900,08:07:30 pm
1,Wednesday,14 February 1900,2,14,1900,2/14/1900,2/14/1900,02:50:12 pm
2,Friday,16 March 1900,3,16,1900,3/16/1900,3/16/1900,09:11:48 am
3,Sunday,15 April 1900,4,15,1900,4/15/1900,4/15/1900,02:02:06 am
4,Monday,14 May 1900,5,14,1900,5/14/1900,5/14/1900,04:36:36 pm


In [15]:
#clean moon data before we merge
full_moon_df = full_moon_df.drop(columns=[' Date old','Month', 'Day.1', 'Year',\
                                          'Concatenate', ' Time'])
#make new column showing full moon date
full_moon_df['Full Moon'] = full_moon_df['date']
full_moon_df.head()

,Day,date,Full Moon
0,Monday,1/15/1900,1/15/1900
1,Wednesday,2/14/1900,2/14/1900
2,Friday,3/16/1900,3/16/1900
3,Sunday,4/15/1900,4/15/1900
4,Monday,5/14/1900,5/14/1900


In [16]:
full_moon_df['date']=pd.to_datetime(full_moon_df['date'])
full_moon_df['Full Moon']=pd.to_datetime(full_moon_df['Full Moon'])
full_moon_df.head()

,Day,date,Full Moon
0,Monday,1900-01-15,1900-01-15
1,Wednesday,1900-02-14,1900-02-14
2,Friday,1900-03-16,1900-03-16
3,Sunday,1900-04-15,1900-04-15
4,Monday,1900-05-14,1900-05-14


In [17]:
#get Row and column count
full_moon_df.dtypes

Day                  object
date         datetime64[ns]
Full Moon    datetime64[ns]
dtype: object

In [18]:
#join the data into a third dataframe
merged_df=gun_violence_df.merge(full_moon_df, on='date',how = "left")

merged_df.head()

,date,state,city_or_county,n_killed,n_injured,incident_characteristics,Day,Full Moon
0,2013-01-01,Pennsylvania,Mckeesport,0,4,Shot - Wounded/Injured||Mass Shooting (4+ vict...,NaN,NaT
1,2013-01-01,California,Hawthorne,1,3,"Shot - Wounded/Injured||Shot - Dead (murder, a...",NaN,NaT
2,2013-01-01,Ohio,Lorain,1,3,"Shot - Wounded/Injured||Shot - Dead (murder, a...",NaN,NaT
3,2013-01-05,Colorado,Aurora,4,0,"Shot - Dead (murder, accidental, suicide)||Off...",NaN,NaT
4,2013-01-07,North Carolina,Greensboro,2,2,"Shot - Wounded/Injured||Shot - Dead (murder, a...",NaN,NaT


In [19]:
merged_df.shape

(239677, 8)

In [23]:
#Write transformed data to csv
merged_df.to_csv('Resources/Crime Data.csv')